In [ ]:
import tensorflow as tf
import pathlib
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
data_dir = "frames"
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

test_data_dir = "test_frames"
test_data_dir = pathlib.Path(test_data_dir)
test_image_count = len(list(test_data_dir.glob('*/*.jpg')))
print(test_image_count)

BATCH_SIZE = 32
IMG_HEIGHT = 720
IMG_WIDTH = 1280
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VALIDATION_STEPS = np.ceil(test_image_count/BATCH_SIZE)

train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
def label_map(datagen):
    for x in datagen:
        
        # Get the filenames that are shuffled
        current_index = ((datagen.batch_index-1) * datagen.batch_size)
        if current_index < 0:
            if datagen.samples % datagen.batch_size > 0:
                current_index = max(0,datagen.samples - datagen.samples % datagen.batch_size)
            else:
                current_index = max(0,datagen.samples - datagen.batch_size)
        index_array = datagen.index_array[current_index:current_index + datagen.batch_size].tolist()
        img_paths = [datagen.filepaths[idx] for idx in index_array]
        y = []
        for f in img_paths:
#             print(f)
            label = np.zeros(2)
            if "-h" in f:
                label[1] = 1
            else:
                label[0] = 1
            y.append(label)
        yield x, np.array(y)

train_data_gen = train_image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode=None)
train_data_gen = label_map(train_data_gen)

test_data_gen = train_image_generator.flow_from_directory(directory=str(test_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode=None)
test_data_gen = label_map(test_data_gen)

data, labels = next(test_data_gen)
# print(labels)
for i in range(len(data)):
    plt.figure()
    plt.imshow(data[i])
    plt.text(1500,200,"label:\n" + str(labels[i]))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(720, 1280, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

In [ ]:
mc = ModelCheckpoint('models/best_non_seq_model_seg.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss=K.binary_crossentropy, 
              optimizer='adam',
              metrics=[])

history = model.fit_generator(train_data_gen, 
                    epochs=20, 
                    validation_data=test_data_gen,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_steps=VALIDATION_STEPS,
                    callbacks=[mc],
                    use_multiprocessing=True,
                    workers=2,
                   )